In [1]:
!docker run -p 9000:9000 -p 9001:9001 \
  -e MINIO_ROOT_USER=minio \
  -e MINIO_ROOT_PASSWORD=minio123 \
  -v $(pwd)/minio_data:/data \
  minio/minio server /data --console-address ":9001"

INFO: Formatting 1st pool, 1 set(s), 1 drives per set.
INFO: WARNING: Host local has more than 0 drives of set. A host failure will result in data becoming unavailable.
MinIO Object Storage Server
Copyright: 2015-2025 MinIO, Inc.
License: GNU AGPLv3 - https://www.gnu.org/licenses/agpl-3.0.html
Version: RELEASE.2025-09-07T16-13-09Z (go1.24.6 linux/arm64)

API: http://172.17.0.2:9000  http://127.0.0.1:9000 
WebUI: http://172.17.0.2:9001 http://127.0.0.1:9001 

Docs: https://docs.min.io
^C


In [1]:
!pip install minio

from minio import Minio
from pathlib import Path


MINIO_ENDPOINT = "127.0.0.1:9000"   
MINIO_USER     = "minio"
MINIO_PASS     = "minio123"
BUCKET         = "model-registry"

client = Minio(MINIO_ENDPOINT, access_key=MINIO_USER, secret_key=MINIO_PASS, secure=False)

if not client.bucket_exists(BUCKET):
    client.make_bucket(BUCKET)

root = Path("model_registry")  
uploaded = []
for p in root.rglob("*"):
    if p.is_file():
        obj = p.relative_to(root).as_posix()   
        client.fput_object(BUCKET, obj, str(p))
        uploaded.append(obj)

print(f"Uploaded {len(uploaded)} objects.")
uploaded[:10]  # show a few


Uploaded 8 objects.


['model1/model.pt',
 'model1/model.joblib',
 'model3/model.pt',
 'model3/model.joblib',
 'model4/model.pt',
 'model4/model.joblib',
 'model2/model.pt',
 'model2/model.joblib']